In [1]:
import numpy as np
from collections import defaultdict as dd
from typing import List, Dict, Set
import nltk
from tqdm import tqdm_notebook, tnrange
import pickle
from termcolor import colored
import sqlite3

In [2]:
import collections

class OrderedSet(collections.MutableSet):

    def __init__(self, iterable=None):
        self.end = end = [] 
        end += [None, end, end]         # sentinel node for doubly linked list
        self.map = {}                   # key --> [key, prev, next]
        if iterable is not None:
            self |= iterable

    def __len__(self):
        return len(self.map)

    def __contains__(self, key):
        return key in self.map

    def add(self, key):
        if key not in self.map:
            end = self.end
            curr = end[1]
            curr[2] = end[1] = self.map[key] = [key, curr, end]

    def discard(self, key):
        if key in self.map:        
            key, prev, next = self.map.pop(key)
            prev[2] = next
            next[1] = prev

    def __iter__(self):
        end = self.end
        curr = end[2]
        while curr is not end:
            yield curr[0]
            curr = curr[2]

    def __reversed__(self):
        end = self.end
        curr = end[1]
        while curr is not end:
            yield curr[0]
            curr = curr[1]

    def pop(self, last=True):
        if not self:
            raise KeyError('set is empty')
        key = self.end[1][0] if last else self.end[2][0]
        self.discard(key)
        return key

    def __repr__(self):
        if not self:
            return '%s()' % (self.__class__.__name__,)
        return '%s(%r)' % (self.__class__.__name__, list(self))

    def __eq__(self, other):
        if isinstance(other, OrderedSet):
            return len(self) == len(other) and list(self) == list(other)
        return set(self) == set(other)

/home/mikolaj/miniconda3/envs/IoT-Pathway/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
conn = sqlite3.connect("Dane/wikipedyjka3.db")

In [4]:
def to_lemma_mapping():
    all_lemmas = dd(list)
    for line in open('Dane/polimorfologik-2.1.txt', encoding='utf-8'):
        L = line.split(';')[:2]
        all_lemmas[L[1].lower()].append(L[0].lower())
    return all_lemmas
lemma_mapping = to_lemma_mapping()

In [5]:
wiki_list = []
key = ""
with open("Dane/fp_wiki.txt") as f:
    cntr = -1
    for line in f:
        tokenized_line = line.split(": ")
        if tokenized_line[0] == "TITLE":
            if tokenized_line[-1][-1] == "\n":
                tokenized_line[-1] = tokenized_line[-1][:-1]
            cntr+=1
            wiki_list.append((tokenized_line[-1], []))
            continue
        wiki_list[-1][1].append(line)

In [42]:
cntr1 = 0
title_positional_index = dd(list)
article_positional_index = dd(list)
title_lemma_positional_index = dd(list)
article_lemma_positional_index = dd(list)

for title, article in wiki_list:
    tokenized_line = nltk.word_tokenize(title)
    cntr2 = 0
    for w in tokenized_line:
        w = w.lower()
        title_positional_index[w].append((cntr1, cntr2))
        for lemma in lemma_mapping[w]:
            title_lemma_positional_index[lemma].append((cntr1, cntr2))
        cntr2 +=1
    cntr2 = 0
    for line in article:
        tokenized_line = nltk.word_tokenize(line)
        for w in tokenized_line:
            w = w.lower()
            article_positional_index[w].append((cntr1, cntr2))
            for lemma in lemma_mapping[w]:
                article_lemma_positional_index[lemma].append((cntr1, cntr2))
            cntr2 += 1
    cntr1 += 1

In [ ]:
c = conn.cursor()
for k, v in title_positional_index.items():
    if not k.isalnum():
        continue
    try:
        c.execute(f'''DROP TABLE IF EXISTS tab_{k}_title''')
        c.execute(f'''CREATE TABLE tab_{k}_title
             (title_position INTEGER, word_position INTEGER)''')
        for ind in v:
            c.execute(f"INSERT INTO tab_{k}_title VALUES ({ind[0]}, {ind[1]})")
    except:
        pass
conn.commit()
for k, v in article_positional_index:
    if not k.isalnum():
        continue
    try:
        c.execute(f'''DROP TABLE IF EXISTS tab_{k}_article''')
        c.execute(f'''CREATE TABLE tab_{k}_article
                 (title_position INTEGER, word_position INTEGER)''')
        for ind in v:
            c.execute(f"INSERT INTO tab_{k}_article VALUES ({ind[0]}, {ind[1]})")
    except:
        pass
conn.commit()
for k, v in title_lemma_positional_index:
    if not k.isalnum():
        continue
    try:
        c.execute(f'''DROP TABLE IF EXISTS tab_{k}_lemma_title''')
        c.execute(f'''CREATE TABLE tab_{k}_lemma_title
                 (title_position INTEGER, word_position INTEGER)''')
        for ind in v:
            c.execute(f"INSERT INTO tab_{k}_lemma_title VALUES ({ind[0]}, {ind[1]})")
    except:
        pass
conn.commit()
for k, v in article_lemma_positional_index:
    if not k.isalnum():
        continue
    try:
        c.execute(f'''DROP TABLE IF EXISTS tab_{k}_lemma_article''')
        c.execute(f'''CREATE TABLE tab_{k}_lemma_article
                 (title_position INTEGER, word_position INTEGER)''')
        for ind in v:
            c.execute(f"INSERT INTO tab_{k}_lemma_article VALUES ({ind[0]}, {ind[1]})")
    except:
        pass
conn.commit()

In [21]:
c = conn.cursor()

In [7]:
def word_query(w):
    w = w.lower()
    res = [OrderedSet(), OrderedSet(), OrderedSet(), OrderedSet()]
    try:
        for el in c.execute(f'SELECT * FROM tab_{w}_title'):
            res[0].add(el[0])
    except:
        pass
    try:
        for el in c.execute(f'SELECT * FROM tab_{w}_article'):
            res[1].add(el[0])
    except:
        pass
    for lemma in lemma_mapping[w]:
        try:
            for el in c.execute(f'SELECT * FROM tab_{lemma}_lemma_title'):
                res[2].add(el[0])
        except:
            pass
        try:
            for el in c.execute(f'SELECT * FROM tab_{lemma}_lemma_article'):
                res[3].add(el[0])
        except:
            pass
    res[1] |= res[0]
    res[2] |= res[1]
    res[3] |= res[2]

#     print(res)
    return res

In [8]:
def phrase_word_query(w, i):
    w = w.lower()
    res = [OrderedSet(), OrderedSet(), OrderedSet(), OrderedSet()]
    try:
        for el in c.execute(f'SELECT * FROM tab_{w}_title'):
            res[0].add((el[0], el[1] - i))
    except:
        pass
    try:
        for el in c.execute(f'SELECT * FROM tab_{w}_article'):
            res[1].add((el[0], el[1] - i))
    except:
        pass
    for lemma in lemma_mapping[w]:
        try:
            for el in c.execute(f'SELECT * FROM tab_{lemma}_lemma_title'):
                res[2].add((el[0], el[1] - i))
        except:
            pass
        try:
            for el in c.execute(f'SELECT * FROM tab_{lemma}_lemma_article'):
                res[3].add((el[0], el[1] - i))
        except:
            pass
    res[1] |= res[0]
    res[2] |= res[1]
    res[3] |= res[2]
    return res

In [47]:
def query(words):
    res = [OrderedSet(), OrderedSet(), OrderedSet(), OrderedSet()]
    for i, w in enumerate(words):
        w = w.lower()
        if i == 0:
            res1 = word_query_alt(w)
            for j in range(4):
                res[j] |= res1[j]
            continue
        res1 = word_query_alt(w)
        for j in range(4):
            res[j] &= res1[j]
    res = phrase_query(words) + res
    for i in range(1,8):
        for j in range(i):
            res[i] -= res[j]
    return res

In [68]:
def word_query_alt(w):
    w = w.lower()
    res = [OrderedSet(), OrderedSet(), OrderedSet(), OrderedSet()]
    try:
        for el in title_positional_index[w]:
            res[0].add(el[0])
    except:
        pass
    try:
        for el in article_positional_index[w]:
            res[1].add(el[0])
    except:
        pass
    for lemma in lemma_mapping[w]:
        try:
            for el in title_lemma_positional_index[lemma]:
                res[2].add(el[0])
        except:
            pass
        try:
            for el in article_lemma_positional_index[lemma]:
                res[3].add(el[0])
        except:
            pass
    res[1] |= res[0]
    res[2] |= res[1]
    res[3] |= res[2]

#     print(res)
    return res

In [49]:
def phrase_query(words):
    res = [OrderedSet(), OrderedSet(), OrderedSet(), OrderedSet()]
    for i, w in enumerate(words):
        w = w.lower()
        if i == 0:
            res1 = phrase_word_query_alt(w, i)
            for j in range(4):
                res[j] |= res1[j]
            continue
        res1 = phrase_word_query_alt(w, i)
        for j in range(4):
            res[j] &= res1[j]
    res1 = res
    res = [OrderedSet(), OrderedSet(), OrderedSet(), OrderedSet()]
    for i in range(4):
        for el in res1[i]:
            res[i].add(el[0])
    res[1] -= res[0]
    res[2] -= res[0] | res[1]
    res[3] -= res[0] | res[1] | res[2]
    return res

In [69]:
def phrase_word_query_alt(w, i):
    w = w.lower()
    res = [OrderedSet(), OrderedSet(), OrderedSet(), OrderedSet()]
    try:
        for el in title_positional_index[w]:
            res[0].add((el[0], el[1] - i))
    except:
        pass
    try:
        for el in article_positional_index[w]:
            res[1].add((el[0], el[1] - i))
    except:
        pass
    for lemma in lemma_mapping[w]:
        try:
            for el in title_lemma_positional_index[lemma]:
                res[2].add((el[0], el[1] - i))
        except:
            pass
        try:
            for el in article_lemma_positional_index[lemma]:
                res[3].add((el[0], el[1] - i))
        except:
            pass
    res[1] |= res[0]
    res[2] |= res[1]
    res[3] |= res[2]
    return res

In [51]:
def to_lemmas(tokens):
    res = []
    for t in tokens:
        t = t.lower()
        res.append(t)
        res += lemma_mapping[t]
    return res

def in_lemmas(w, lemmas):
    w = w.lower()
    if w in lemmas:
        return True
    for l in lemma_mapping[w]:
        if l in lemmas:
            return True
    return False


def search_and_print(words):
    is_phrase = words[0] == "\""
    if is_phrase:
        words = words[1:-1]
    tokens = [t.lower() for t in nltk.word_tokenize(words)]
    article_sets = phrase_query(tokens) if is_phrase else query(tokens)
    lemmas = to_lemmas(tokens)
    for articles in article_sets:
        for el in articles:
            title, article = wiki_list[el]
            for w in nltk.word_tokenize(title):
                if in_lemmas(w, lemmas):
                    print(colored(w, "green"), end=" ")
                else:
                    print(w, end=" ")
            print("\n")
            for line in article:
                for w in nltk.word_tokenize(line):
                    if in_lemmas(w, lemmas):
                        print(colored(w, "green"), end=" ")
                    else:
                        print(w, end=" ")
                print("")
            print("")

In [52]:
print(word_query("wysoki"))

[OrderedSet(), OrderedSet(), OrderedSet(), OrderedSet()]


In [78]:
search_and_print('kodeks karny')

Kodeks karny 

Kodeks karny 
Kodeks karny ( skrót to `` k.k . `` , w języku prawniczym przyjął się również `` kk `` ) – akt normatywny stanowiący zbiór przepisów regulujących odpowiedzialność karną obywateli danego państwa . Określa definicję przestępstwa , zasady odpowiedzialności za przestępstwo ( w tym okoliczności wyłączające bezprawność czynu – tzw . kontratypy ) , zasady wymiaru kary , zasady przedawnienia odpowiedzialności karnej . Ustala także katalog kar , innych środków przymusu oraz środki związane z poddaniem sprawcy próbie ( jak warunkowe zawieszenie wykonania kary ) . 


Kodeks karny wojskowy 

Kodeks karny wojskowy 
Kodeks karny wojskowy – akt normatywny stanowiący zbiór przepisów regulujących prawo karne wojskowe . 
W Polsce osobna ustawa dla prawa karnego wojskowego obowiązywała w okresie od 1918 do 1970 . W 1970 wszedł w życie Kodeks karny z 1969 , który zawierał w części trzeciej przepisy odnoszące się do żołnierzy . Obecnie w Polsce nie ma kodeksu karnego wojskowego

In [74]:
search_and_print('cień mgły')

Carlos Ruiz Zafón 

Carlos Ruiz Zafón 
Carlos Ruiz Zafón ( ur . 25 września 1964 w Barcelonie ) – hiszpański pisarz . 
Z wykształcenia jest dziennikarzem . Do swoich ulubionych autorów Zafón zalicza Dostojewskiego , Tołstoja i Dickensa . Od roku 1993 mieszka w Los Angeles , gdzie poświęcił się pisaniu scenariuszy filmowych i powieściopisarstwu . Jego pierwsze cztery książki były adresowane do młodzieży ( za `` Książę mgły `` otrzymał nagrodę Edebé – ) . Kolejne trzy książki ( `` Cień wiatru `` , `` Gra anioła `` i `` Więzień nieba `` ) stanowią trylogię . Ich fabuła jest bardziej rozwinięta w stosunku do poprzednich książek , przez co są skierowane również do starszych czytelników . Książka `` La sombra del Viento `` ( `` Cień wiatru `` ) , wydana po raz pierwszy w roku 2001 , została do roku 2008 przetłumaczona na ponad 45 języków i sprzedana w ponad 10 milionach egzemplarzy , m.in . w Niemczech , Brazylii , Danii , Francji , Włoszech , Finlandii , Portugalii , Stanach Zjednoczonych ,

In [67]:
lemma_mapping["ciągła"]

['ciągło', 'ciągły']